In [1]:
import wandb
import torch
import torch.nn as nn
import torchvision.transforms as T
import timm

from train import FashionTrainer, config, set_seed, model_size

## Baseline

In [2]:
WANDB_PROJECT = "fmnist_bench"
WANDB_ENTITY = "capecape"

In [3]:
config

namespace(epochs=5,
          model_name='resnet10t',
          bs=512,
          device='cuda',
          seed=42,
          lr=0.001,
          use_wandb=True,
          wd=0.0)

In [4]:
set_seed(config.seed)

In [5]:
mean, std = (0.28, 0.35)

In [6]:
train_tfms = T.Compose([
    T.RandomCrop(28, padding=1), 
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean, std),
    # T.RandomErasing(scale=(0.02, 0.25), value="random"),
])

val_tfms = T.Compose([
    T.ToTensor(),
    T.Normalize(mean, std),
])

tfms = {"train": train_tfms, "valid":val_tfms}

## Train a custom ConvMixer

In [7]:
from timm.models.convmixer import ConvMixer

In [8]:
model_params = {"kernel_size":5, "dim":256, "depth":10, "patch_size":1, "in_chans":1, "num_classes":10}
module = ConvMixer(**model_params)
module.head = nn.Sequential(
    nn.Dropout(0.2), 
    nn.Flatten(), 
    nn.Linear(model_params["dim"], model_params["num_classes"], bias=False), 
    nn.BatchNorm1d(model_params["num_classes"])
)
config.model_name = "convmixer"
config.model_params = model_params
config.model_size = model_size(module)
config.lsuv = False

In [9]:
config.epochs = 5
config.lr = 1e-2
config.bs = 128

model = FashionTrainer(module, bs=config.bs, tfms=tfms)

model.compile(epochs=config.epochs, lr=config.lr, wd=config.wd)

In [10]:
# train
run = wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY, config=config)
model.fit()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape. Use `wandb login --relogin` to force relogin


epoch:   0, train_loss:      0.545, train_acc: 0.842   ||   val_loss:      0.512, val_acc: 0.895


epoch:   1, train_loss:      0.274, train_acc: 0.906   ||   val_loss:      0.276, val_acc: 0.899


epoch:   2, train_loss:      0.221, train_acc: 0.924   ||   val_loss:      0.220, val_acc: 0.924


epoch:   3, train_loss:      0.176, train_acc: 0.940   ||   val_loss:      0.176, val_acc: 0.936


epoch:   4, train_loss:      0.133, train_acc: 0.955   ||   val_loss:      0.138, val_acc: 0.942


In [11]:
wandb.finish()

epoch,▁▃▅▆█
learning_rate,▂▂▅▇██████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train_acc,▁▅▆▇█
train_loss,██▆▅▄▂▃▃▃▃▃▃▂▂▂▃▂▂▁▃▃▁▂▂▃▁▂▂▂▂▁▂▁▃▂▁▁▁▁▁
val_acc,▁▂▅▇█
val_loss,█▄▃▂▁
epoch,4
learning_rate,0.0
train_acc,0.95517
train_loss,0.19313
val_acc,0.9415


=============================

## Power off the instance


In [ ]:
!sudo poweroff